# 🤖 OpenAI Agents SDK – Class 7 (4 October 2025)
# Handsoff

### 📦 Install `openai-agents` package

In [ ]:
%pip install -qU openai-agents python-dotenv

### Make Jupyter Notebook capable of running asynchronous functions.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

### 🌐 Global Configurations for Connecting to Gemini

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from openai import AsyncOpenAI
from agents import set_default_openai_api, set_default_openai_client, set_tracing_disabled

load_dotenv(find_dotenv())
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Which LLM Service?
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)


# Global Setup for Connecting to Gemini
set_tracing_disabled(disabled=True)
set_default_openai_client(client=external_client)
set_default_openai_api("chat_completions")


### 📜 Enable verbose to debug every etep

In [ ]:
from agents import enable_verbose_stdout_logging

enable_verbose_stdout_logging()

### 🤝 Triage Agent that handsoff

In [ ]:
from agents import Agent, Runner, handoff
import asyncio

billing_agent = Agent(name="Billing agent", instructions="Handle billing questions.", model="gemini-2.5-flash")
refund_agent  = Agent(name="Refund agent",  instructions="Handle refunds.", model="gemini-2.5-flash")

triage_agent = Agent(
    name="Triage agent",
    instructions=(
        "Help the user with their questions. "
        "If they ask about billing, handoff to the Billing agent. "
        "If they ask about refunds, handoff to the Refund agent."
    ),
    model="gemini-2.5-flash",
    handoffs=[billing_agent, handoff(refund_agent)],  # either direct agent or `handoff(...)`
)

async def main():
    result = await Runner.run(triage_agent, "I need to check refund status.")
    print('='*50, '\n')
    print(result.final_output)
    print(result.last_agent)

asyncio.run(main())
